# Optiguide Analysis

Ashwin Kandath, Gareth Minson-Efimov, Raj Shah

In this notebook, we explore Optiguide to gather managerial insights from our optimization model

## Interface Setup

In [7]:
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import re
import requests  # for loading the example source code
import openai
import os


import autogen
from autogen.agentchat import Agent, UserProxyAgent
from optiguide import OptiGuideAgent

In [8]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-4",
            "gpt-4o",
            "gpt4",
            "gpt-4-32k",
            "gpt-4-32k-0314",
            "gpt-3.5-turbo",
            "gpt-3.5-turbo-16k",
            "gpt-3.5-turbo-0301",
            "chatgpt-35-turbo-0301",
            "gpt-35-turbo-v0301",
        }
    }
)

In [9]:
# Define the path to the local .ipynb file
file_path = "optiguide_script.py"

# Open and read the local .ipynb file
try:
    with open(file_path, 'r') as file:
        code = file.read()
except FileNotFoundError:
    raise RuntimeError(f"Failed to find the file at {file_path}")

# Show the first and last 10 lines of the source code
print("\n".join(code.split("\n")))
# print(".\n" * 3)
# print("\n".join(code.split("\n")[-10:]))


import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd


# Loading Dataset
data = pd.read_csv('2026_full_data.csv')

time_periods = data.columns[26:].values
num_time_periods = len(time_periods)
precinct_num = data.shape[0]
clusters = data['cluster'].values
cluster_dict = {0: 3, 1: 5, 2: 3, 3: 5, 4: 5, 5: 3}
arrival_rates = data.iloc[:, 26:].values  # Shape: [precinct_num][num_time_periods]

# Service rates
service_rate_check_in = 30 / 12  # 2.5 voters per 5-minute period
service_rate_voting = 60 / 12  # 5 voters per 5-minute period

# Resource limits
total_check_in_stations = precinct_num * 3
total_voting_booths = precinct_num * 9
max_check_in_stations = 5
max_voting_booths = 10

# Create a new model
m = gp.Model("MinimizeQueueWaiting")

# OPTIGUIDE DATA CODE GOES HERE

C = m.addVars(precinct_num, vtype=GRB.INTEGER, lb=2, ub=max_check_in_stations, name="CheckInStations")
V = m.addVars(precinct_num, vtype=GRB.INTEGER, lb=3, ub=max_voting_booths, name="Voti

In [10]:
# In-context learning examples.
example_qa = """
----------
Question: what happens if we change max_voting_booths = 10 to max_voting_booths = 5 while keeping max_check_in_stations = 5??
Answer Code:
```python
max_check_in_stations = 5 
max_voting_booths = 5
```

----------
Question: What will happen if the minimum number of checking stations is 2?
Answer Code:
```python
C = model.addVars(precinct_num, vtype=GRB.INTEGER, lb=2, ub=max_check_in_stations, name="CheckInStations")
V = model.addVars(precinct_num, vtype=GRB.INTEGER, lb=3, ub=max_voting_booths, name="VotingBooths")
```

"""

In [11]:
agent = OptiGuideAgent(name="OptiGuide",source_code=code,debug_times=3, example_qa = example_qa, llm_config={"seed": 42,"config_list": config_list,})
user = UserProxyAgent("user", max_consecutive_auto_reply=0,human_input_mode="NEVER", code_execution_config=False)

Set parameter MIPGap to value 0.035
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.1.0 24B91)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 652034 rows, 328280 columns and 1464808 nonzeros
Model fingerprint: 0xfe919320
Variable types: 0 continuous, 328280 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+00, 1e+01]
  RHS range        [1e+00, 1e+04]
Presolve removed 571755 rows and 246601 columns
Presolve time: 0.84s
Presolved: 80279 rows, 81679 columns, 241434 nonzeros
Variable types: 0 continuous, 81679 integer (2471 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.01s

Solved with dual simplex

Root relaxation: objective 6.014834e+05, 25678 iterations, 0.59 seconds (1.12 work units)

    Nodes    |    Current Node    |     Objecti

## Prompting

We prompt a series of questions to gather responses on various considerations for a decision maker.

In [17]:
ans1 = user.initiate_chat(agent, message="Right now, to assist the disenchranchised clusters (1,3,4), we are setting a higher lower bound for the number of voting booths by calculating min_booth_voting. How will the objective function change if we do the same for check-in stations, instead of voting booths?")
print(ans1.summary)

user (to OptiGuide):

Right now, to assist the disenchranchised clusters (1,3,4), we are setting a higher lower bound for the number of voting booths by calculating min_booth_voting. How will the objective function change if we do the same for check-in stations, instead of voting booths?

--------------------------------------------------------------------------------
OptiGuide (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide):

```python
# We will need to modify the dictionary cluster_dict to be as per the minimum for check-in stations
# For illustration purposes lets assume we should have a minimum of 2 check-in stations for cluster 1, 3, and 4 
min_check_in_stations_dict = {0: 1, 1: 2, 2: 1, 3: 2, 4: 2, 5: 1}

# Then, we will add the constraints:
for i in range(precinct_num):
    min_check_in_stations = min_check_in_stations_dict[clusters[i]]
    m.addConstr(C[i] >= min_check_in_stations, name=f"MinCh

In [19]:
ans2 = user.initiate_chat(agent, message="How can dynamic reallocation (based on predicted peak times) improve overall system performance?")
print(ans2.summary)

user (to OptiGuide):

How can dynamic reallocation (based on predicted peak times) improve overall system performance?

--------------------------------------------------------------------------------
OptiGuide (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide):

```python
# First, we need to predict peak times for each precinct. This can be done using time series forecasting methods,
# for which we'd need historical data. Assuming that we have predictions in the peak_times list:

peak_times = [...] # list of predicted peak times for each precinct

# Then, during non-peak times, we can dynamically reallocate the extra resources to precincts about to experience peak times.
# We add the following dynamic reallocation constraint to the model

for t in range(num_time_periods-1):
    for i in range(precinct_num):
        if t not in peak_times:
            m.addConstr(C[i,t] + V[i,t] <= C[i,t+1] + V[i,t+1], na

In [20]:
ans3 = user.initiate_chat(agent, message="How would increases or decreases in voter turnout in priority precincts impact the optimal allocation strategy?")
print(ans3.summary)

user (to OptiGuide):

How would increases or decreases in voter turnout in priority precincts impact the optimal allocation strategy?

--------------------------------------------------------------------------------
OptiGuide (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide):

```python
# Assume we have information about turnout rates for each precinct in a list called turnout_rates
# and we adjust the arrival rates of voters based on these rates.
# We can assume that the turnout rates are fractional values between 0 and 1.

turnout_rates = np.random.uniform(0.5, 1.0, precinct_num) # this is for example

# Adjust the arrival rates
arrival_rates = data.iloc[:, 26:].values * np.expand_dims(turnout_rates, axis=-1) 

# The rest of the code remains the same. The solver will find the new optimal allocation
# of checking stations and voting booths based on the adjusted voter arrival rates.
```

----------------

In [21]:
ans4 = user.initiate_chat(agent, message="If turnout predictions are off by 10%, how do total wait times and resource allocations shift?")
print(ans4.summary)

user (to OptiGuide):

If turnout predictions are off by 10%, how do total wait times and resource allocations shift?

--------------------------------------------------------------------------------
OptiGuide (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide):

```python
# Increase arrival rates by 10%
arrival_rates = data.iloc[:, 26:].values * 1.1  # Shape: [precinct_num][num_time_periods]
```

--------------------------------------------------------------------------------
OptiGuide (to safeguard):


--- Code ---
# Increase arrival rates by 10%
arrival_rates = data.iloc[:, 26:].values * 1.1  # Shape: [precinct_num][num_time_periods]

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to OptiGuide):

SAFE

--------------------------------------------------------------------------------
Set parameter MIPGap to value 0.035
G

In [26]:
ans4 = user.initiate_chat(agent, message=" If additional resources were available, where should they be allocated to achieve the greatest reduction in wait times? COnsidering doubling the number of checkin stations")
print(ans4.summary)

user (to OptiGuide):

 If additional resources were available, where should they be allocated to achieve the greatest reduction in wait times? COnsidering doubling the number of checkin stations

--------------------------------------------------------------------------------
OptiGuide (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide):

```python
total_check_in_stations = precinct_num * 6  # double the number of check-in stations
```

--------------------------------------------------------------------------------
OptiGuide (to safeguard):


--- Code ---
total_check_in_stations = precinct_num * 6  # double the number of check-in stations

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to OptiGuide):

SAFE

--------------------------------------------------------------------------------
Set parameter MIPGap to value 0.0

In [29]:
ans5 = user.initiate_chat(agent, message="In our implementation, do you think we have missed out on something important, that can possibly alter the outcome of the model? If yes, give is concrete actionables to improve the model")
print(ans5.summary)

user (to OptiGuide):

In our implementation, do you think we have missed out on something important, that can possibly alter the outcome of the model? If yes, give is concrete actionables to improve the model

--------------------------------------------------------------------------------
OptiGuide (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide):

```python
# The model currently does not consider the change in arrival rates over time. 
# Arrivals rates are taken from the data, but does not take into account any variability or underlying trends in the data. 
# This could be improved by modeling the arrival rates as a stochastic process rather than static values.

# Further, model also does not consider the distribution of voters among precincts. 
# It assumes that voters are evenly distributed which may not be the case in real situations. 
# A data-driven approach could be used to estimate the distribu